In [ ]:
#| default_exp config

## Config Manager

A class to fetch required config from AWS Systems Manager.
> Contans information such as S3 bucket used for datalake and database addresses

>Note that this is not used for secrets

In [ ]:
#| exporti
#| hide

import boto3

In [ ]:
#| export
#| hide

class ConfigManager:
    """
    A class to fetch required config for LakeInterface from AWS Systems Manager.
    ...

    Attributes
    ----------
    manager : boto3 session client
        AWS Systems Manager client
        
    Methods
    -------
    
    __init__(profile='default'):
        Initializes the AWS SSM session using AWS profile_name
        
    fetch_config(config_group):
        Uses manager to fetch parameters from the /config_group path in AWS SSM
    
    """
    
    def __init__(self, profile=None):
        if profile:
            session = boto3.session.Session(profile_name=profile)
        else:
            session = boto3.session.Session()
            
        self.manager = session.client('ssm')

    def _get_parameter(self, config_group, parameter_name):
        return self.manager.get_parameter(Name=parameter_name)['Parameter']['Value']

    def fetch_config(self, config_group):
        response = self.manager.describe_parameters(
            ParameterFilters=[
                {
                    'Key': 'Path',
                    'Values': [
                        f'/{config_group}',
                    ]
                }
            ]
        )

        return dict((
            p['Name'].replace(f'/{config_group}/',''), 
            self._get_parameter(config_group, p['Name'])
        ) for p in response['Parameters'])


In [ ]:

#cfg_manager = ConfigManager(profile='personal')
cfg_manager = ConfigManager()
cfg_manager.fetch_config('example')

{'param1': 'foo', 'param2': 'bar'}